In [33]:
import ast
import urllib2
import urllib
from cStringIO import StringIO
import json
import geopandas as gpd
import pandas as pd
import shapely
from shapely.wkt import loads as load_wkt

In [34]:
SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql'

def queryCartoDB(query, format = 'CSV', source = SQL_SOURCE):
    """ queries a url with an sql compatible search
    Arguments:
    query : the sql query string
    format : the expected format
    source " the url of the API
    """
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return StringIO(response.read())

In [35]:
# fill in the query details in facilityQuery: 
# within a single query collect AND GROUP the data by zipcode
facilityQuery = '''
SELECT
    count(*) as count,
    zipcodes
FROM nycasthmasdismissals 
GROUP BY        
    zipcodes
ORDER BY count DESC
'''

facilityStream = queryCartoDB(facilityQuery, format='JSON')
facilityData   = json.loads(facilityStream.read())
#facilityData
facilities     = pd.DataFrame.from_dict(facilityData['rows'])
facilities

,count,zipcodes
0,2164,10467
1,1829,11203
2,1699,10461
3,1698,10451
4,1156,11237
5,1072,10456
6,940,11201
7,926,11040
8,822,10457
9,713,11212


In [36]:
facilities['zipcodes'] = facilities['zipcodes'].astype(str)

In [37]:
import os

In [38]:
from geopandas import GeoDataFrame
shape_path=os.getenv('PUIDATA')+"/doitt_zip_code_01_29jul2009.shp"
zip_shape = GeoDataFrame.from_file(shape_path)
zip_shape.head()

,bldgzip,cartodb_id,complaints,county,created_at,crime,cty_fips,geometry,po_name,shape_area,shape_leng,st_fips,state,updated_at,zip
0,0,86,None,Queens,2015-03-08,57.0,081,POLYGON ((-73.78663963052189 40.79032165698663...,Bayside,3.883482e+07,31490.883733,36,NY,2015-03-08,11360.0
1,0,87,None,Queens,2015-03-08,164.0,081,"POLYGON ((-73.89671713692546 40.7829420395804,...",Astoria,4.879561e+07,36787.574255,36,NY,2015-03-08,11105.0
2,0,76,None,Bronx,2015-03-08,351.0,005,"POLYGON ((-73.9317564471804 40.80793457778948,...",Bronx,3.106361e+07,25984.026841,36,NY,2015-03-08,10454.0
3,0,84,None,Queens,2015-03-08,153.0,081,"POLYGON ((-73.85464575618116 40.7894326573659,...",College Point,4.332116e+07,39905.989552,36,NY,2015-03-08,11356.0
4,0,85,None,Queens,2015-03-08,1.0,081,POLYGON ((-73.78177608464604 40.79484590011937...,Bayside,6.567982e+06,12841.612895,36,NY,2015-03-08,11359.0


In [39]:
zip_shape.rename(columns={'zip':'zipcodes'}, inplace=True)

In [40]:
zip_shape.head()

,bldgzip,cartodb_id,complaints,county,created_at,crime,cty_fips,geometry,po_name,shape_area,shape_leng,st_fips,state,updated_at,zipcodes
0,0,86,None,Queens,2015-03-08,57.0,081,POLYGON ((-73.78663963052189 40.79032165698663...,Bayside,3.883482e+07,31490.883733,36,NY,2015-03-08,11360.0
1,0,87,None,Queens,2015-03-08,164.0,081,"POLYGON ((-73.89671713692546 40.7829420395804,...",Astoria,4.879561e+07,36787.574255,36,NY,2015-03-08,11105.0
2,0,76,None,Bronx,2015-03-08,351.0,005,"POLYGON ((-73.9317564471804 40.80793457778948,...",Bronx,3.106361e+07,25984.026841,36,NY,2015-03-08,10454.0
3,0,84,None,Queens,2015-03-08,153.0,081,"POLYGON ((-73.85464575618116 40.7894326573659,...",College Point,4.332116e+07,39905.989552,36,NY,2015-03-08,11356.0
4,0,85,None,Queens,2015-03-08,1.0,081,POLYGON ((-73.78177608464604 40.79484590011937...,Bayside,6.567982e+06,12841.612895,36,NY,2015-03-08,11359.0


In [41]:
zip_shape['zipcodes'] = zip_shape['zipcodes'].astype(int)

In [45]:
zip_shape.head()

,bldgzip,cartodb_id,complaints,county,created_at,crime,cty_fips,geometry,po_name,shape_area,shape_leng,st_fips,state,updated_at,zipcodes
0,0,86,None,Queens,2015-03-08,57.0,081,POLYGON ((-73.78663963052189 40.79032165698663...,Bayside,3.883482e+07,31490.883733,36,NY,2015-03-08,11360
1,0,87,None,Queens,2015-03-08,164.0,081,"POLYGON ((-73.89671713692546 40.7829420395804,...",Astoria,4.879561e+07,36787.574255,36,NY,2015-03-08,11105
2,0,76,None,Bronx,2015-03-08,351.0,005,"POLYGON ((-73.9317564471804 40.80793457778948,...",Bronx,3.106361e+07,25984.026841,36,NY,2015-03-08,10454
3,0,84,None,Queens,2015-03-08,153.0,081,"POLYGON ((-73.85464575618116 40.7894326573659,...",College Point,4.332116e+07,39905.989552,36,NY,2015-03-08,11356
4,0,85,None,Queens,2015-03-08,1.0,081,POLYGON ((-73.78177608464604 40.79484590011937...,Bayside,6.567982e+06,12841.612895,36,NY,2015-03-08,11359


In [46]:
zip_shape['zipcodes'] = zip_shape['zipcodes'].astype(str)

In [48]:
##merge
facilitydb = pd.merge(facilities, zip_shape, how='inner', on=['zipcodes'])
facilitydb.drop(['bldgzip', 'county', 'cty_fips', 'po_name',
       'state', 'st_fips','cartodb_id','complaints','created_at','crime','updated_at'], axis=1, inplace=True)
facilitydb.head()

,count,zipcodes,geometry,shape_area,shape_leng
0,2164,10467,POLYGON ((-73.88010836736842 40.89519554426504...,6.933617e+07,43944.750630
1,1829,11203,POLYGON ((-73.94269628450726 40.66405515298856...,6.064484e+07,32799.751802
2,1699,10461,POLYGON ((-73.84218235210301 40.83537416906358...,6.282406e+07,35943.350556
3,1698,10451,POLYGON ((-73.93293579565668 40.82772456126229...,2.894411e+07,32573.914438
4,1156,11237,POLYGON ((-73.92816524502244 40.71420826250719...,2.737378e+07,30772.617425
